In [ ]:
!pip install flask-ngrok

In [ ]:
import numpy as np
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import re
from nltk.tokenize import WordPunctTokenizer
from keras.models import Sequential
from keras import layers
from keras import models
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

In [ ]:
data['review'] = data['review'].str.replace('<br />', ' ')
data['review'] = data['review'].str.replace('\d+', '')
data['review'] = data['review'].str.lower()
data['review'] = data['review'].str.replace('.', '')
data['review'] = data['review'].str.replace('-', '')

In [ ]:
tok = WordPunctTokenizer()
symbols = r'@[A-Za-z0-9]+'
url = r'https?://[A-Za-z0-9./]+'
remove = r'|'.join((symbols, url))

In [ ]:
def cleaning_data(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    clean = re.sub(remove, '', souped)
    letters = re.sub("[^a-zA-Z]", ' ', clean)
    lower = letters.lower()
    words = tok.tokenize(lower)
    return (" ".join(words)).strip()

In [ ]:
clean_df = data

In [ ]:
for i in range(0,data.shape[0]):                                                                  
    clean_df['review'][i] = cleaning_data(data['review'][i])
    clean_df['sentiment'][i] = 1 if data['sentiment'][i] == 'positive' else 0

In [ ]:
X = clean_df['review'].values
y = clean_df['sentiment'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test  = vectorizer.transform(X_test)

In [ ]:
import pickle
pickle.dump(vectorizer, open('/content/drive/MyDrive/textvectorizer1.pkl','wb'))

In [ ]:
input_dim = X_train.shape[1] 

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',  optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                1152400   
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,152,411
Trainable params: 1,152,411
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, np.array(y_train).astype('float32'), epochs=10,verbose=False,validation_data=(X_test, np.array(y_test).astype('float32')), batch_size=10)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:450: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 10), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


In [ ]:
loss, accuracy = model.evaluate(X_train,  np.array(y_train).astype('float32'), verbose=False)
print("Training Accuracy: ",accuracy)
loss, accuracy = model.evaluate(X_test, np.array(y_test).astype('float32'), verbose=False)
print("Testing Accuracy:  ",accuracy)

Training Accuracy:  0.9997714161872864
Testing Accuracy:   0.8730666637420654


In [ ]:
model.save('/content/drive/MyDrive/reviewmodel1.h5')

In [5]:
!pip install flask-ngrok
import numpy as np
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import re
from nltk.tokenize import WordPunctTokenizer
from keras.models import Sequential
from keras import layers
from keras import models
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
import pickle

In [6]:
tok = WordPunctTokenizer()
symbols = r'@[A-Za-z0-9]+'
url = r'https?://[A-Za-z0-9./]+'
remove = r'|'.join((symbols, url))

In [7]:
def cleaning_data(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    clean = re.sub(remove, '', souped)
    letters = re.sub("[^a-zA-Z]", ' ', clean)
    lower = letters.lower()
    words = tok.tokenize(lower)
    return (" ".join(words)).strip()

In [8]:
loaded = models.load_model('/content/drive/MyDrive/reviewmodel1.h5')
vectorizer = pickle.load(open('/content/drive/MyDrive/textvectorizer1.pkl','rb'))

In [10]:
app = Flask(__name__)
run_with_ngrok(app)


@app.route("/", methods=["POST"])
def home():
  data = request.json
  review = data['review']
  review = cleaning_data(review)
  review = [review]
  review = vectorizer.transform(review)
  score  = (loaded.predict((review))[0][0]*5).round()
  return str(score)

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8679-34-74-62-36.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Nov/2021 06:33:56] "POST / HTTP/1.1" 200 -
